In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import joblib
import datetime
import os
import snowflake.connector
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
# import lightgbm and xgboost 
import lightgbm as lgb 
from lightgbm import LGBMRegressor
import xgboost as xgb 
from xgboost import plot_importance
from matplotlib import pyplot

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score

#RMSE/MAE computation using sklearn library
from sklearn.metrics import mean_squared_error, mean_absolute_error
import scipy
from xgboost import XGBRegressor
from IPython.display import display, HTML
from sklearn.model_selection import GridSearchCV
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from statistics import mean
from itertools import product

# Predict RET values

In [3]:
pd_query="""
select * from  scratch.riders.final_prediction_feat_set 
"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )

cur = conn.cursor()
cur.execute(pd_query)
df = cur.fetch_pandas_all()




Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJdb9owFIb%2FSuRdJ3YCo2AREINWzcYKg1Bpu%2FNiEwyOnfmDlH8%2FExqpu2gvdhEpcp5z%2FJy8Zzx9qURwZtpwJVMQRwgETBaKclmmYJc%2FhEMQGEskJUJJloILM2A6GRtSiRrPnD3IDfvjmLGBbyQNbj%2BkwGmJFTHcYEkqZrAt8Hb2fYmTCOFaK6sKJcCbko8riDFMW2%2FYlVDDvd7B2hpD2DRN1PQipUuYIIQgGkFPXZFPHf%2FiZ3qHjyHqX3lPeHz96vaFy9sv%2BEjr9w0y%2BDHP1%2BF6tc1BMOtU50oaVzG9ZfrMC7bbLG8Cxhss7pfZ8%2F1mtYqYCwsmrSYijCMjVbMX5MQKVdXO%2BtaRf4N7RqFQJffTZ4sU1CdO5%2BcEZbOvMc%2BPrrrsLgNZ7sTTtncqL%2FKx%2FOaa%2BrCnP%2B6Oo%2BpnAYLnLt7kGm9mjGOZvIZq%2FRFK%2BiG6C5Nejob48wD3hxEajX6BYOFD5ZLYtrIzJ0WhnLQmKpUqBWv9FLwmmEBO66l%2FOE3nKD7K02hQgNum4PZOPfmv%2BcfwbYvXzXvyYWSLtRK8uAQPSlfEvp9VHMXtCafhvkUxqwgXM0o1M8ZnJoRq5poR6xfcascAnNxu%2FXfFJ38B&RelayState=62246

In [4]:
print(df.shape)
df.sample(1)


(40341, 29)


,ZONE_CODE,CITY_NAME,HOUR_OF_DAY,WEEK_OF_DAY,LATE_BY_5_MINS_PERC_LAST_14_DYS,P50_RESTO_CUSTOMER_MINS_LAST_90_DYS,AVG_WAIT_AT_CUSTOMER_LAST_90_DYS,AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS,AVG_TEMPERATURE_LAST_90_DYS,P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS,...,P70_RESTO_CUSTOMER_MINS_LAST_14_DYS_SAME_HR,AVG_TEMPERATURE_LAST_14_DYS_SAME_HR,AVG_TEMPERATURE_LAST_5_DYS_SAME_HR,AVG_WAIT_AT_CUSTOMER_LAST_3_DYS_SAME_HR,P50_EOD_LAST_180_DYS_SAME_HR_WOD,P20_RIDER_TO_RESTAURANT_MINS_LAST_180_DYS_SAME_HR_WOD,P10_ERAT_LAST_90_DYS_SAME_HR_WOD,P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD,P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD,P10_FPT_LAST_5_DYS_SAME_HR_WOD
36090,AAS,Kuwait,19,Fri,0.203485,10.0,4.13815,3.40323,32.268922,0.0,...,13.0,38.70247,38.760968,3.868333,26.508333,1.0,363.4,13.0,0.0,3.799


##  Load the model

In [5]:
#  Load the 
xgb_model=joblib.load('/Users/richakumari/Desktop/Clustering/RET_prediction/RETfinal_v1_1.pkl')

In [6]:
cols=[    
'P50_AOD_LAST_14_DYS_SAME_HR',
'AVG_FPT_LAST_21_DYS',
'P70_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS_SAME_HR',
'AVG_ERAT_LAST_14_DYS_SAME_HR',
'AVG_ERAT_LAST_90_DYS_SAME_HR',
'P50_EOD_LAST_180_DYS_SAME_HR_WOD',
'AVG_WAIT_AT_CUSTOMER_LAST_3_DYS_SAME_HR',
'AVG_ERAT_LAST_5_DYS',
'P50_RESTO_CUSTOMER_MINS_LAST_90_DYS',
'P90_AOD_LAST_180_DYS_SAME_HR',
'AVG_WAIT_AT_CUSTOMER_LAST_90_DYS',
'AVG_ERAT_LAST_21_DYS_SAME_HR',
'P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS',
'LATE_BY_5_MINS_PERC_LAST_14_DYS',
'P70_RESTO_CUSTOMER_MINS_LAST_14_DYS_SAME_HR',
'P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD',
'P10_ERAT_LAST_90_DYS_SAME_HR_WOD',
'LATE_BY_10_MINS_PERC_LAST_3_DYS',
'AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS',
'P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD',
'P20_RIDER_TO_RESTAURANT_MINS_LAST_180_DYS_SAME_HR_WOD',
'AVG_TEMPERATURE_LAST_90_DYS',
'AVG_TEMPERATURE_LAST_14_DYS_SAME_HR',
'AVG_TEMPERATURE_LAST_5_DYS_SAME_HR',
'P10_FPT_LAST_5_DYS_SAME_HR_WOD',
]


In [7]:
#  Predict the Result 
x = df[cols]
df['RET_pred'] =xgb_model.predict(x)


In [8]:
#  caping the abnormally high and low valus
p999=df.groupby('CITY_NAME')['RET_pred'].transform(lambda x: x.quantile(0.999))
df['RET_pred'] = df['RET_pred'].clip(upper=p999)

# Cap values lower than 0 in RET_AVG to 5
df.loc[df['RET_pred'] <=5, 'RET_pred'] = 5


# obtaining the UR values here 

In [9]:
#  in case AOD is null replace it by city average AOD for same time
AOD_avg ="""with base as(
select 
distinct 
lower(city_name) as city_name
,case when zone_code in
('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
when zone_code in (
'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
else city_name end as Cluster_Assigned
,case 
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak Lunch'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
     else 'Other' end as shift
,round(COALESCE(SUM(aod_mins_sum ), 0) / NULLIF(COALESCE(SUM(aod_mins_cnt ), 0), 0),2) AS AOD_AVG
    
from
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where  date (start_of_period_local)  between  dateadd('month', -1, date_trunc('month',  current_date())) 
and LAST_DAY(dateadd('month', -1, date_trunc('month',  current_date())), MONTH)
and is_within_zone_hours  -- use this filter to get only active zone

group by 1,2,3
)
,map as(
 select
 distinct 
 city_name
 ,zone_code
 ,case when zone_code in
('ABAR','AKW','AMR','AMZ','ANH','AQ','AQPP','AQS','ATW','AWH','DBB','DCH','DD11','DDO','DEIR','DFC','DIC','DMEY','DSO',
'DSW','DT','DWS','FCOW','HCC','HPDT','JMH','KAR','MRDF','NAS','RAK','UMS','WAA','ZBL') then  'Deira and Downtown Cluster'
when zone_code in (
'AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM') then 'Sportcity and Marina Cluster'
else city_name end as Cluster_Assigned
,case 
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (12, 13) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu','Fri') then 'Peak Lunch'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Mon', 'Tue', 'Wed','Thu') then 'Peak Dinner'
     when CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT)  in (19, 20) and DAYNAME(START_OF_PERIOD_LOCAL) in ('Fri','Sat','Sun') then 'Super Peak Dinner'
     else 'Other' end as shift
FROm
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where  date (start_of_period_local) between   dateadd('month', -1, date_trunc('month',  current_date())) 
    and LAST_DAY(dateadd('month', -1, date_trunc('month',  current_date())), MONTH)
and is_within_zone_hours
) 

select 
map.*, base. AOD_AVG
from 
map
left join
base 
on map.Cluster_Assigned=base.Cluster_Assigned
and map.shift=base.shift


"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(AOD_avg)
AOD_df = cur.fetch_pandas_all()





Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJNb%2BIwEIb%2FSuQ9J3bS8lGLgCgEGokWxEdXuzfjGOrFsbO2Q2B%2F%2FToBpPbQHnqwZNnvzDMz7%2FQGp1x4R6YNVzIGYYCAxyRVGZf7GGzWE78LPGOJzIhQksXgzAwY9HuG5KLAw9K%2BySX7WzJjPZdIGlx%2FxKDUEitiuMGS5MxgS%2FFq%2BDzDUYAwMYZp63DgGpIZ7lhv1hYYwqqqguouUHoPI4QQRA%2FQqWrJD%2FAOUXzNKLSyiipxCzm5nj5BhBDd1wincITFNfCRy8sIvqJsLyKDn9brhb%2BYr9bAG966GylpypzpFdNHTtlmObsUYFwF42SWvibL%2BTxgpU%2BZtJoIPwyMVNVOkAOjKi9K61IH7gZ3LINC7bkbWDqOQXHg2eNmmrSey9O%2FY2v2s7M6J60JOZ35NmknkqbT8a%2FtUzvcTlfbDaXAe73ZG9X2psaULJW1qdY9oejeRx0%2FulujLm51MHKidvc38MbOVC6JbSJvlRNKVSmtCfZK7QVr6lOwdiSCPCsG7vAsHqHwjzw8tCm4bApumLr%2Frf578H2K6%2Ba9ODPS8UIJTs%2FeROmc2M%2B9CoOweeGZv2ukmOWEi2GWaWaM80wIVY00I9YtuNUlA7B%2FoX5c8f5%2F&RelayState=62262

In [10]:
print(AOD_df.shape)
AOD_df.sample(2)

(1067, 5)


,CITY_NAME,ZONE_CODE,CLUSTER_ASSIGNED,SHIFT,AOD_AVG
614,Sharjah,SUBSANDC,Sharjah,Peak Dinner,35.03
864,Dubai,DM,Sportcity and Marina Cluster,Peak Lunch,35.28


##  Load the UR model equation

In [11]:
# #  read the csv with the equations for each city
# current_directory = os.getcwd()
# file_name = 'eqn.csv'
# file_path = os.path.join(current_directory, file_name)

UR_eqn = pd.read_csv('/Users/richakumari/Desktop/Clustering/RET_prediction/eqn_all_cities_july_vf.csv')
UR_eqn.head(1)


,Unnamed: 0,CLUSTER_ASSIGNED,Equation,MAE,MAPE,R2,RMSE,Overprediction %,Underprediction %
0,0,Kuwait,UR_ZONE_CORRECTED = -0.5527 + 0.0323 * AOD_AVG,0.040501,0.069032,0.763835,0.052642,43.865031,56.134969


In [12]:
clusters_to_expand_deira = ['ABAR', 'AKW', 'AMR', 'AMZ', 'ANH', 'AQ', 'AQPP', 'AQS', 'ATW', 'AWH', 'DBB', 'DCH', 'DD11',
                      'DDO', 'DEIR', 'DFC', 'DIC', 'DMEY', 'DSO', 'DSW', 'DT', 'DWS', 'FCOW', 'HCC', 'HPDT', 'JMH',
                      'KAR', 'MRDF', 'NAS', 'RAK', 'UMS', 'WAA', 'ZBL']

clusters_to_expand_sportcity =['AR','DBRS','DDH','DEGC','DHS','DHS2','DICP','DIP','DJP','DM','DSB','DTS','HPJLT','HPMC','IBN','JBLI','JGE','JI',
'JLT','JVC','JVT','MDW8','PC','PR','RMRM','SC','SPR','SS','TM','TWM']
# Function to expand clusters
def expand_clusters(row):
    clusters_assigned = row['CLUSTER_ASSIGNED'].split(', ')  # Split if multiple clusters are present
    if 'Deira and Downtown Cluster' in clusters_assigned :
        expanded_rows = []
        for cluster in clusters_to_expand_deira:
            new_row = {'CLUSTER_ASSIGNED': cluster, 'Equation': row['Equation']}
            expanded_rows.append(new_row)
        return pd.DataFrame(expanded_rows)
    if 'Sportcity and Marina Cluster' in clusters_assigned :
        expanded_rows = []
        for cluster in clusters_to_expand_sportcity:
            new_row = {'CLUSTER_ASSIGNED': cluster, 'Equation': row['Equation']}
            expanded_rows.append(new_row)
        return pd.DataFrame(expanded_rows)
    else:
        return pd.DataFrame([row])

# Apply the function to each row and concatenate results
expanded_df = pd.concat([expand_clusters(row) for _, row in UR_eqn.iterrows()], ignore_index=True)


UR_eqn_df=expanded_df[['Equation','CLUSTER_ASSIGNED']]



def extract_components(equation):
    try:
        # Split by '=' to separate left-hand side and right-hand side
        lhs, rhs = equation.split('=')
        
        # Strip whitespace from both sides
        lhs = lhs.strip()
        rhs = rhs.strip()
        
        # Check if rhs contains '+'
        if '+' in rhs:
            # Split by '+' to separate constant and term with variable
            constant, term_with_var = [part.strip() for part in rhs.split('+')]
            
            # Extract variable and coefficient
            coefficient, variable = [part.strip() for part in term_with_var.split('*')]
            
            # Strip any additional whitespace from variable
            variable = variable.strip()
            
            # Convert constants to float
            constant = float(constant)
            coefficient = float(coefficient)
            
            # Return a Series with components
            return pd.Series([lhs, constant, coefficient, variable], index=['LHS', 'Constant', 'Coefficient', 'Variable'])
        else:
            raise ValueError("Unexpected format in equation: {}".format(equation))
    
    except Exception as e:
        print(f"Error processing equation '{equation}': {str(e)}")
        return pd.Series([None, None, None, None], index=['LHS', 'Constant', 'Coefficient', 'Variable'])

# Apply the function to each row and expand into new columns
UR_eqn_df[['LHS', 'Constant', 'Coefficient', 'Variable']] = UR_eqn_df['Equation'].apply(extract_components)
UR_eqn_df. head(2)


,Equation,CLUSTER_ASSIGNED,LHS,Constant,Coefficient,Variable
0,UR_ZONE_CORRECTED = -0.5527 + 0.0323 * AOD_AVG,Kuwait,UR_ZONE_CORRECTED,-0.5527,0.0323,AOD_AVG
1,UR_ZONE_CORRECTED = -0.6321 + 0.0299 * AOD_AVG,Doha,UR_ZONE_CORRECTED,-0.6321,0.0299,AOD_AVG


In [13]:
#  zones which do not have UR but have is_within_zone true, they are missing from here, approx 31 zones 
# list['ADMV',	'DBI',	'AQ2',	'AQ3',	'BK',	'BKP',	'BRSS',	'DBS',	'EH',	'JH',	'PCG',	'PCW',	'TL',	'DDD',	'DGT',	'DIFC',	'DML',	'DPB',	'DXBA',	'DXBAA',	'DXBB',	'DXBC',	'ET',	'IT',	'JBL',	'MAJ',	'PKPK',	'RAH',	'SAB',	'STW',	'SZ1',	'SZ2']

In [14]:
# Conditionally merge based on city
# Option 1: When city is 'Dubai', merge on cluster_id and cluster_assigned

condition_dubai = df['CITY_NAME'] == 'Dubai'
merged_dubai = pd.merge(df[condition_dubai], UR_eqn_df, left_on='ZONE_CODE', right_on='CLUSTER_ASSIGNED')

# Option 2: When city is not 'Dubai', merge on city and cluster_assigned
condition_not_dubai = df['CITY_NAME'] != 'Dubai'
merged_not_dubai = pd.merge(df[condition_not_dubai], UR_eqn_df, left_on='CITY_NAME', right_on='CLUSTER_ASSIGNED')

# Concatenate the results from both merges
final_merged_df = pd.concat([merged_dubai, merged_not_dubai])
final_merged_df.sample(2)

,ZONE_CODE,CITY_NAME,HOUR_OF_DAY,WEEK_OF_DAY,LATE_BY_5_MINS_PERC_LAST_14_DYS,P50_RESTO_CUSTOMER_MINS_LAST_90_DYS,AVG_WAIT_AT_CUSTOMER_LAST_90_DYS,AVG_RIDER_TO_RESTAURANT_MINS_LAST_90_DYS,AVG_TEMPERATURE_LAST_90_DYS,P10_RIDER_TO_RESTAURANT_MINS_LAST_30_DYS,...,P70_RESTO_CUSTOMER_MINS_LAST_90_DYS_SAME_HR_WOD,P10_RIDER_TO_RESTAURANT_MINS_LAST_14_DYS_SAME_HR_WOD,P10_FPT_LAST_5_DYS_SAME_HR_WOD,RET_pred,Equation,CLUSTER_ASSIGNED,LHS,Constant,Coefficient,Variable
4879,TWM,Dubai,14,Thu,0.163636,13.0,4.543797,6.31203,33.209358,2.0,...,14.8,3.0,NaN,22.193827,UR_ZONE_CORRECTED = -0.7073 + 0.0379 * AOD_AVG,TWM,UR_ZONE_CORRECTED,-0.7073,0.0379,AOD_AVG
1197,KAZ,Kuwait,9,Wed,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,32.335190,UR_ZONE_CORRECTED = -0.5527 + 0.0323 * AOD_AVG,Kuwait,UR_ZONE_CORRECTED,-0.5527,0.0323,AOD_AVG


In [15]:
# Define a function to calculate Peak_Type
def calculate_peak_type(row):
    hour = row['HOUR_OF_DAY']
    weekday = row['WEEK_OF_DAY']
    
    if hour in [12, 13] and weekday in ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']:
        return 'Peak Lunch'
    elif hour in [19, 20] and weekday in ['Mon', 'Tue', 'Wed', 'Thu']:
        return 'Peak Dinner'
    elif hour in [19, 20] and weekday in ['Fri', 'Sat', 'Sun']:
        return 'Super Peak Dinner'
    else:
        return 'Other'

# Apply the function to create the new column 'SHIFT'
final_merged_df['SHIFT'] = final_merged_df.apply(calculate_peak_type, axis=1)


#  based on zone X shift populate the AOD values 
final_merged_df = pd.merge(final_merged_df, AOD_df[['ZONE_CODE', 'SHIFT', 'AOD_AVG']], on=['ZONE_CODE', 'SHIFT'], how='left')


In [16]:
final_merged_df['UR']=final_merged_df['Constant']+final_merged_df['Coefficient'].astype(float)*final_merged_df['AOD_AVG'].astype(float)

# Calculate the mean UR for each CITY_NAME and SHIFT group
mean_ur_by_city_shift = final_merged_df.groupby(['CITY_NAME', 'SHIFT'])['UR'].transform('mean')

# Fill NaN values in UR with the calculated mean values
final_merged_df['UR_updated'] = final_merged_df['UR'].fillna(mean_ur_by_city_shift)

#  case when UR is nan  replace with avg of the shift X city

final_merged_df['pred_TP']=final_merged_df['UR_updated']*(60/final_merged_df['RET_pred'])

In [17]:
# Define the mapping for WEEK_OF_DAY for normal cities
week_day_mapping = {'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7}

# Define the mapping for WEEK_OF_DAY for Kuwait and Doha
week_day_mapping_kw_qt = {'Sun': 1, 'Mon': 2, 'Tue': 3, 'Wed': 4, 'Thu': 5, 'Fri': 6, 'Sat': 7}

# Adjust WEEK_OF_DAY based on the city
def adjust_week_day(row):
    if row['CITY_NAME'] in ['Kuwait', 'Doha']:
        adjusted_day_numeric = week_day_mapping_kw_qt[row['WEEK_OF_DAY']]
    else:
        adjusted_day_numeric = week_day_mapping[row['WEEK_OF_DAY']]
    
    return adjusted_day_numeric

# Apply the function to the dataframe
final_merged_df['ADJUSTED_WEEK_OF_DAY_NUMERIC'] = final_merged_df.apply(adjust_week_day, axis=1)

download=final_merged_df[['ZONE_CODE','ADJUSTED_WEEK_OF_DAY_NUMERIC','HOUR_OF_DAY','pred_TP','SHIFT']]

download = download.rename(columns={
    'ZONE_CODE': 'zone_code',
    'ADJUSTED_WEEK_OF_DAY_NUMERIC': 'iso_weekday',
    'HOUR_OF_DAY': 'hour',
    'pred_TP': 'throughput_target',

})


In [18]:
x= download[(download['throughput_target'].isna() | (download['throughput_target'] == ''))]
x


,zone_code,iso_weekday,hour,throughput_target,SHIFT


#  get the base and old TP values

In [19]:
old_TP="""select 
 distinct 
 zone_code
 ,zone_id
 ,country_name
 ,city_name
 ,CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT) as hour_of_day
 ,(DATE_PART('DAYOFWEEK', START_OF_PERIOD_LOCAL)+1) as DAY_OF_WEEK
 ,round(sum(agg_zone_delivery_metrics_hourly.v_hat_raw_0day) / nullif(sum(agg_zone_delivery_metrics_hourly.r_star_raw_0day), 0),2) AS throughput_target_0_day
from 
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where
date (start_of_period_local) between   current_date-30 and current_date
and is_within_zone_hours 
and country_name in ('Kuwait','Qatar','United Arab Emirates')
group by 1,2,3,4,5 ,6

"""

conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(old_TP)
old_TP_df = cur.fetch_pandas_all()



Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZJdb9owFIb%2FSuRdJ3bSshSLgPgoGlpbGFCYdmdih3o4dubjNPDvZ0KRuov2YheRIuc5x8%2FJe3qDY6mCV2FBGp2hOCIoEDo3XOp9hp7X0%2FAOBeCY5kwZLTJ0EoAG%2FR6wUlV0WLsXvRR%2FagEu8I000PZDhmqrqWEggWpWCqAup6vh4wNNIkIra5zJjULvSj6vYADCOm94LeEgvd6LcxXFuGmaqLmJjN3jhBCCSRd76ox8ufJHP9MHfIzJ7Zn3hMcXb24jqS%2B%2F4DOt3QUC%2Bm29XoSL%2BWqNguFVdWw01KWwK2FfZS6elw8XAfAGk%2FuH2eZ%2BOZ9Hog5zoZ1lKowj0KYpFDuI3JRV7XzryL%2FhQnCszF766WeTDFUHySXEj2kxSk9y2h2l2933nwDb2P2QfHfaFgVZWHWUm3TKOoccBZtrvMk53hlALWb6HKrzRyS5DUkaJjdrckc7KU1IlJDOLxRMfKhSM9dWXs1ZnptaO4j2xuyVaP0MPieYYMmrgX8kz8Yk%2Fq0P3a85umwKbe%2B0%2Ff%2Bav4fft3jbvCcfxmyyMErmp2BqbMncx1nFUdyeSB4WLUpFyaQacm4FgM9MKdOMrWDOL7iztUC4f7n13xXv%2FwU%3D&RelayState=62274 to authentica

In [20]:
print(old_TP_df.shape)
print(old_TP_df.THROUGHPUT_TARGET_0_DAY.sum())

# Rename columns in old_TP_df to match fnal column name  for joining
old_TP_df.rename(columns={'ZONE_ID':'zone_id','ZONE_CODE': 'zone_code', 'DAY_OF_WEEK': 'iso_weekday', 'HOUR_OF_DAY': 'hour'}, inplace=True)
old_TP_df.sample()

(40405, 7)
43283.13


,zone_code,zone_id,COUNTRY_NAME,CITY_NAME,hour,iso_weekday,THROUGHPUT_TARGET_0_DAY
26378,AAY,3328,United Arab Emirates,Al Ain,17,3,0.91


In [21]:
print(download.shape)
print(download.throughput_target.sum())
download.sample()


(36225, 5)
46827.302291337095


,zone_code,iso_weekday,hour,throughput_target,SHIFT
21811,QATLUAIB,4,22,1.093511,Other


In [22]:
#  join the 2 data set with the base as left data on zone_code, week and hour and keep both TP old vs TP new
master_file = pd.merge(old_TP_df, download, on=['zone_code', 'iso_weekday', 'hour'], how='left')
master_file = master_file[['zone_id','CITY_NAME','zone_code', 'iso_weekday', 'hour',  'THROUGHPUT_TARGET_0_DAY', 'throughput_target','SHIFT']]


master_file=master_file[master_file['CITY_NAME']!='Al Wakrah']# filter out 'Al Wakrah



In [23]:
#  filtering for one city for testing purpose
# master_file=master_file[master_file['CITY_NAME']=='Ras Al Khaimah']
# master_file.shape
# master_file.THROUGHPUT_TARGET_0_DAY.sum()

In [24]:
#  fill na in shift columns

shift_mapping_df = master_file.dropna(subset=['SHIFT']).drop_duplicates(subset=['CITY_NAME', 'iso_weekday', 'hour', 'SHIFT'])

# Create a dictionary for quick lookup
mapping_dict = shift_mapping_df.set_index(['CITY_NAME', 'iso_weekday', 'hour'])['SHIFT'].to_dict()

# Function to fill null SHIFT values using the mapping dictionary
def fill_shift(row):
    if pd.isnull(row['SHIFT']):
        return mapping_dict.get((row['CITY_NAME'], row['iso_weekday'], row['hour']), None)
    return row['SHIFT']

# Apply the function to fill null SHIFT values
master_file['SHIFT'] = master_file.apply(fill_shift, axis=1)


In [25]:
# Calculate the average throughput_target for each combination of ZONE_CODE, HOUR_OF_DAY, and DAY_OF_WEEK and Replace the null values in throughput_target with the calculated averages
avg_throughput_target = master_file.groupby(['CITY_NAME', 'iso_weekday', 'SHIFT'])['throughput_target'].transform('mean')

master_file['throughput_target'].fillna(avg_throughput_target, inplace=True)
master_file['throughput_target']=master_file['throughput_target'].round(2)


                   

#  Added steps for Doha to take care of summer ban -- remove this post summer ban (september)

In [26]:
# Replace peak lunch values with peak dinner values for doha
mask_peak_dinner = (master_file['CITY_NAME'] == 'Doha') & (master_file['SHIFT'] == 'Peak Dinner')

# Calculate the average throughput_target for Peak Dinner at zone_code X iso_weekday level
peak_dinner_avg = master_file[mask_peak_dinner].groupby(['zone_code', 'iso_weekday'])['throughput_target'].mean().reset_index()

# Convert to dictionary for easier lookup
peak_dinner_mapping = peak_dinner_avg.set_index(['zone_code', 'iso_weekday'])['throughput_target'].to_dict()

# Create a mask for Doha city and Peak Lunch shift
mask_peak_lunch = (master_file['CITY_NAME'] == 'Doha') & (master_file['SHIFT'] == 'Peak Lunch')

# Replace throughput_target for Peak Lunch with the corresponding average Peak Dinner throughput_target
master_file.loc[mask_peak_lunch, 'throughput_target'] = master_file[mask_peak_lunch].apply(
    lambda row: peak_dinner_mapping.get((row['zone_code'], row['iso_weekday']), row['throughput_target']),
    axis=1
)
#  replace the Friday's peak lunch data with thursday's peak lunch data

mask_weekday_5_lunch = (master_file['CITY_NAME'] == 'Doha') & (master_file['iso_weekday'] == 5) & (master_file['SHIFT'] == 'Peak Lunch')

# Create a mapping for iso_weekday = 5 Peak Lunch values
weekday_5_lunch_mapping = master_file[mask_weekday_5_lunch].set_index(['zone_code', 'hour'])['throughput_target'].to_dict()
len(weekday_5_lunch_mapping)
# Create a mask for iso_weekday = 6 and Peak Lunch shift
mask_weekday_6_lunch = (master_file['CITY_NAME'] == 'Doha') & (master_file['iso_weekday'] == 6) & (master_file['SHIFT'] == 'Peak Lunch')

# Replace throughput_target for iso_weekday = 6 Peak Lunch with the corresponding iso_weekday = 5 Peak Lunch throughput_target
master_file.loc[mask_weekday_6_lunch, 'throughput_target'] = master_file[mask_weekday_6_lunch].apply(
    lambda row: weekday_5_lunch_mapping.get((row['zone_code'], row['hour']), row['throughput_target']),
    axis=1
)



# Download the final csv

In [37]:
Final_download_file=master_file[['zone_id','iso_weekday','hour','throughput_target']]
# Get the current month name
current_month = datetime.datetime.now().strftime("%B")

# Create a file name with the current month
file_name = f'TP_all_cities_{current_month}.csv'

# Save to CSV
Final_download_file.to_csv(file_name, index=False)





In [28]:
master_file[(master_file['throughput_target'].isna())]

,zone_id,CITY_NAME,zone_code,iso_weekday,hour,THROUGHPUT_TARGET_0_DAY,throughput_target,SHIFT


In [39]:
print(Final_download_file.shape)
Final_download_file.throughput_target.sum()

(39845, 4)


51635.51

#  QC steps to check if the new prediction vs old prediction are close or far

In [29]:
#  obtain the old and new values at city X cluster X shift level weighted by ov

cluster_df="""select 
 distinct 
 country_name
 ,city_name
 ,CLUSTER_NAME
 ,zone_code
 ,zone_id
 ,CAST(EXTRACT(HOUR FROM START_OF_PERIOD_LOCAL) AS INT) as hour_of_day
 ,(DATE_PART('DAYOFWEEK', START_OF_PERIOD_LOCAL)+1) as DAY_OF_WEEK
 , COALESCE(SUM(orders_delivered ), 0) as orders_delivered 
 ,round(sum(agg_zone_delivery_metrics_hourly.v_hat_raw_0day) / nullif(sum(agg_zone_delivery_metrics_hourly.r_star_raw_0day), 0),2) AS throughput_target_0_day
from 
PRODUCTION.AGGREGATE.AGG_ZONE_DELIVERY_METRICS_HOURLY
where 
date (start_of_period_local) between   current_date-30 and current_date
and is_within_zone_hours 
and country_name in ('Kuwait','Qatar','United Arab Emirates')

group by 1,2,3,4,5 ,6,7"""


conn = snowflake.connector.connect(
        user='Richa.kumari@deliveroo.com',
        password=os.getenv('password'),
        account="deliveroo.eu-central-1",
        authenticator='externalbrowser',
        warehouse="BI_DEVELOPMENT",
        role="BI_DEVELOPMENT",
    )


cur = conn.cursor()
cur.execute(cluster_df)
cluster_df = cur.fetch_pandas_all()



Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://accounts.google.com/o/saml2/idp?idpid=C01jnk96c&SAMLRequest=nZLBbuIwEIZfJfKeEztJKcUiIApUy4otFGh3tTfjGHBx7NTjNPTt1yRF6h7awx4iRc4342%2FyT394KlTwKixIozMURwQFQnOTS73P0OPmLrxBATimc6aMFhl6E4CGgz6wQpV0VLmDXomXSoALfCMNtPmQocpqahhIoJoVAqjjdD36OadJRGhpjTPcKPSh5OsKBiCs84aXkhyk1zs4V1KM67qO6jQydo8TQggmPeypM%2FLtwp%2F8TJ%2FwMSZXZ94THl%2B%2Bu91K3f6Cr7S2LQT0%2B2azDJeL9QYFo4vq2GioCmHXwr5KLh5X81YAvMFkOp89TVeLRSSqkAvtLFNhHIE29U6xo%2BCmKCvnW0f%2BDe9EjpXZSz%2F9bJKh8ijz3Xq%2B%2Ff3QuT6Z%2BNe0syqKbXX7THbl6EGuZLoFff9DH9Ljy5YQjoKnS7zJOd4ZQCVm%2Bhyq80ckuQpJN0zSDbmhnS5Nk6jXTf6gYOJDlZq5pvJizjg3lXYQ7Y3ZK9H4GXxOMMEyL4f%2BkXk2JvGzPvauOWo3hTZ32sF%2Fzd%2FHH1u8b969D2M2WRol%2BVtwZ2zB3OdZxVHcnMg83DUoFQWTapTnVgD4zJQy9dgK5vyCO1sJhAftrf%2Bu%2BOAv&RelayState=62288 to authen

In [30]:
cluster_df.sample()
print(master_file.THROUGHPUT_TARGET_0_DAY.sum())
print(master_file.throughput_target.sum())
print(master_file.shape)

43283.130000000005
51635.51
(39845, 8)


In [31]:
# Merge the DataFrames on 'zone_code', 'iso_weekday', and 'hour'
qc_df = pd.merge(master_file, cluster_df, left_on=['zone_code', 'iso_weekday', 'hour'], right_on=['ZONE_CODE', 'DAY_OF_WEEK', 'HOUR_OF_DAY'], how='left')

# Select the desired columns
result_df = qc_df[['zone_id', 'CITY_NAME_x', 'zone_code', 'iso_weekday', 'hour', 'THROUGHPUT_TARGET_0_DAY_x', 'throughput_target', 'SHIFT',  'CLUSTER_NAME', 'ORDERS_DELIVERED']]

# Rename columns for clarity
result_df.rename(columns={
    'CITY_NAME_x': 'CITY_NAME',
    'THROUGHPUT_TARGET_0_DAY_x': 'THROUGHPUT_TARGET_0_DAY',
}, inplace=True)

result_df.head(1)

,zone_id,CITY_NAME,zone_code,iso_weekday,hour,THROUGHPUT_TARGET_0_DAY,throughput_target,SHIFT,CLUSTER_NAME,ORDERS_DELIVERED
0,3440,Dubai,DSB,6,7,1.38,1.43,Other,AR-DBRS-DBS-DDH-DEGC-DHS-DHS2-DICP-DIP-DJP-DM-...,24


In [32]:
# Function to calculate weighted average for a group
def calculate_weighted_avg(group, value_column, weight_column):
    return (group[value_column] * group[weight_column]).sum() / group[weight_column].sum()

# Group by CITY_NAME and calculate the weighted averages for old and new throughput
weighted_avg_city = result_df.groupby('CITY_NAME').apply(lambda x: pd.Series({
    'weighted_avg_old': calculate_weighted_avg(x, 'THROUGHPUT_TARGET_0_DAY', 'ORDERS_DELIVERED'),
    'weighted_avg_new': calculate_weighted_avg(x, 'throughput_target', 'ORDERS_DELIVERED')
}))


print(weighted_avg_city)


# Group by CITY_NAME and calculate the weighted averages for old and new throughput
weighted_avg_city_shift_cluster = result_df.groupby(['CITY_NAME', 'SHIFT','CLUSTER_NAME']).apply(lambda x: pd.Series({
    'weighted_avg_old': calculate_weighted_avg(x, 'THROUGHPUT_TARGET_0_DAY', 'ORDERS_DELIVERED'),
    'weighted_avg_new': calculate_weighted_avg(x, 'throughput_target', 'ORDERS_DELIVERED')
})).reset_index()


weighted_avg_city_shift_cluster.to_csv('weighted_avg_city_shift_cluster.csv', index=False)

print(weighted_avg_city_shift_cluster)

                weighted_avg_old  weighted_avg_new
CITY_NAME                                         
Abu Dhabi               1.412603          1.451532
Ajman                   1.624720          1.274884
Al Ain                  0.987261          0.898582
Doha                    1.221237          1.146721
Dubai                   1.404619          1.442684
Kuwait                  1.442995          1.470481
Ras Al Khaimah          1.355226          1.388829
Sharjah                 1.657732          1.376665
     CITY_NAME              SHIFT  \
0    Abu Dhabi              Other   
1    Abu Dhabi        Peak Dinner   
2    Abu Dhabi         Peak Lunch   
3    Abu Dhabi  Super Peak Dinner   
4        Ajman              Other   
..         ...                ...   
170    Sharjah         Peak Lunch   
171    Sharjah         Peak Lunch   
172    Sharjah  Super Peak Dinner   
173    Sharjah  Super Peak Dinner   
174    Sharjah  Super Peak Dinner   

                                          CLU

In [33]:
#  print the avg TP at city X shift level both old and new
avg_tp_df = master_file.groupby(['CITY_NAME', 'SHIFT'])[['THROUGHPUT_TARGET_0_DAY', 'throughput_target']].mean().reset_index()
avg_tp_df['perc_diff']=(avg_tp_df['throughput_target'] - avg_tp_df['THROUGHPUT_TARGET_0_DAY']) / avg_tp_df['THROUGHPUT_TARGET_0_DAY'] * 100
avg_tp_df

def highlight_diff(s):
    return ['background-color: red' if v > 15 or v < -15 else '' for v in s]

# Apply the function to the `abs_diff` column
styled_df = avg_tp_df.style.apply(highlight_diff, subset=['perc_diff'])
red_rows_count = avg_tp_df[(avg_tp_df['perc_diff'] > 15) | (avg_tp_df['perc_diff'] < -15)].shape[0]

print(f'Number of rows highlighted in red: {red_rows_count}')
print(avg_tp_df.shape)
styled_df


# Other		1.17
# Peak Dinner 	1.42
# Peak_lunch		1.02
# Super Peak Dinner		1.17


Number of rows highlighted in red: 10
(32, 5)


,CITY_NAME,SHIFT,THROUGHPUT_TARGET_0_DAY,throughput_target,perc_diff
0,Abu Dhabi,Other,1.284193,1.308730,1.910722
1,Abu Dhabi,Peak Dinner,1.375060,1.422500,3.450067
2,Abu Dhabi,Peak Lunch,1.331476,1.371000,2.968420
3,Abu Dhabi,Super Peak Dinner,1.421032,1.496667,5.322536
4,Ajman,Other,1.416400,1.100086,-22.332242
5,Ajman,Peak Dinner,1.476500,1.138750,-22.875042
6,Ajman,Peak Lunch,1.442400,1.136545,-21.204558
7,Ajman,Super Peak Dinner,1.508833,1.143485,-24.213972
8,Al Ain,Other,0.914655,0.918509,0.421301
9,Al Ain,Peak Dinner,0.970893,0.846000,-12.863712


In [34]:
#  print the avg TP at city X shift level both old and new
avg_tp_df_week = master_file.groupby(['CITY_NAME', 'SHIFT','iso_weekday'])[['THROUGHPUT_TARGET_0_DAY', 'throughput_target']].mean().reset_index()
avg_tp_df_week['perc_diff']=(avg_tp_df_week['throughput_target'] - avg_tp_df_week['THROUGHPUT_TARGET_0_DAY']) / avg_tp_df_week['THROUGHPUT_TARGET_0_DAY'] * 100


def highlight_diff(s):
    return ['background-color: red' if v > 15 or v < -15 else '' for v in s]

# Apply the function to the `abs_diff` column
styled_df = avg_tp_df_week.style.apply(highlight_diff, subset=['perc_diff'])
red_rows_count = avg_tp_df_week[(avg_tp_df_week['perc_diff'] > 15) | (avg_tp_df_week['perc_diff'] < -15)].shape[0]

print(f'Number of rows highlighted in red: {red_rows_count}')
print(avg_tp_df_week.shape)

styled_df

Number of rows highlighted in red: 47
(152, 6)


,CITY_NAME,SHIFT,iso_weekday,THROUGHPUT_TARGET_0_DAY,throughput_target,perc_diff
0,Abu Dhabi,Other,1,1.282381,1.303305,1.631681
1,Abu Dhabi,Other,2,1.282381,1.298319,1.242874
2,Abu Dhabi,Other,3,1.282381,1.311933,2.304449
3,Abu Dhabi,Other,4,1.282381,1.306947,1.915642
4,Abu Dhabi,Other,5,1.282381,1.311120,2.241104
5,Abu Dhabi,Other,6,1.288321,1.309173,1.618551
6,Abu Dhabi,Other,7,1.288170,1.319048,2.396980
7,Abu Dhabi,Peak Dinner,1,1.465714,1.420714,-3.070175
8,Abu Dhabi,Peak Dinner,2,1.344048,1.411190,4.995571
9,Abu Dhabi,Peak Dinner,3,1.345238,1.430000,6.300885


 # ----------------------------------------------END ---------------------------------------

In [35]:

# # Define the mapping for WEEK_OF_DAY for normal cities
# week_day_mapping = {'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6, 'Sun': 7}
# reverse_week_day_mapping = {v: k for k, v in week_day_mapping.items()}

# # Define the mapping for WEEK_OF_DAY for Kuwait and Qatar
# week_day_mapping_kw_qt = {'Sun': 1, 'Mon': 2, 'Tue': 3, 'Wed': 4, 'Thu': 5, 'Fri': 6, 'Sat': 7}
# reverse_week_day_mapping_kw_qt = {v: k for k, v in week_day_mapping_kw_qt.items()}

# # Adjust HOUR_OF_DAY and WEEK_OF_DAY based on the conditions
# def adjust_hour_and_day(row):
#     if row['CITY_NAME'] in ['Kuwait', 'Doha']:
#         if 0 <= row['HOUR_OF_DAY'] <= 6:
#             adjusted_hour = row['HOUR_OF_DAY'] + 24
#             adjusted_day_numeric = week_day_mapping_kw_qt[row['WEEK_OF_DAY']] - 1
#             if adjusted_day_numeric == 0:
#                 adjusted_day_numeric = 7
#             adjusted_day = reverse_week_day_mapping_kw_qt[adjusted_day_numeric]
#         else:
#             adjusted_hour = row['HOUR_OF_DAY']
#             adjusted_day = row['WEEK_OF_DAY']
#         adjusted_day_numeric = week_day_mapping_kw_qt[adjusted_day]
#     else:
#         if 0 <= row['HOUR_OF_DAY'] <= 6:
#             adjusted_hour = row['HOUR_OF_DAY'] + 24
#             adjusted_day_numeric = week_day_mapping[row['WEEK_OF_DAY']] - 1
#             if adjusted_day_numeric == 0:
#                 adjusted_day_numeric = 7
#             adjusted_day = reverse_week_day_mapping[adjusted_day_numeric]
#         else:
#             adjusted_hour = row['HOUR_OF_DAY']
#             adjusted_day = row['WEEK_OF_DAY']
#         adjusted_day_numeric = week_day_mapping[adjusted_day]
    
#     return pd.Series([adjusted_hour, adjusted_day, adjusted_day_numeric])

# final_merged_df[['ADJUSTED_HOUR_OF_DAY', 'ADJUSTED_WEEK_OF_DAY', 'ADJUSTED_WEEK_OF_DAY_NUMERIC']] = final_merged_df.apply(adjust_hour_and_day, axis=1)

